In [45]:
from PIL import Image
import numpy as np
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def read_and_process_images(base_dir):
    images = []
    labels = []  # Collecting labels
    label_dict = {}  # A dictionary to map subfolder names to unique labels
    label_counter = 0

    # Iterate through all subdirectories
    for subdir, dirs, files in os.walk(base_dir):
        for file in files:
            # Construct the full file path
            filepath = os.path.join(subdir, file)
            try:
                # Open the image
                with Image.open(filepath) as img:
                    # Convert to RGB
                    img = img.convert('RGB')
                    # Resize to 32x32
                    img = img.resize((32, 32), Image.ANTIALIAS)
                    # Convert to numpy array and scale
                    img_array = np.asarray(img, dtype=np.float32) / 255
                    images.append(img_array)
                    # Assign label based on the subdirectory name
                    label = subdir.split(os.path.sep)[-1]
                    if label not in label_dict:
                        label_dict[label] = label_counter
                        label_counter += 1
                    labels.append(label_dict[label])
            except IOError:
                # Handle the case where the file could not be opened as an image
                print(f"Cannot open {file} as an image.")
                
    return images, labels

base_dir = r"C:\Users\shaif\Downloads\Compressed\CUB_200_2011\CUB_200_2011\images"
X_train, YT = read_and_process_images(base_dir)

# Convert YT to categorical
num_classes = len(np.unique(YT))
YT_categorical = to_categorical(YT, num_classes=num_classes)
from sklearn.utils import shuffle
X_train, YT_categorical = shuffle(X_train, YT_categorical)

# Perform train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, YT_categorical, test_size=0.2, random_state=42)

C:\Users\shaif\AppData\Local\Temp\ipykernel_19088\1941233865.py:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


In [46]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
#base_model = MobileNet(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  

In [47]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train).astype('float32'), Y_train, batch_size=64, epochs=100, validation_split = .2)

Epoch 1/100
118/118 [==============================] - 16s 58ms/step - loss: 5.6966 - accuracy: 0.0137 - val_loss: 7.6640 - val_accuracy: 0.0037
Epoch 2/100
118/118 [==============================] - 6s 47ms/step - loss: 5.1256 - accuracy: 0.0338 - val_loss: 9.8491 - val_accuracy: 0.0037
Epoch 3/100
118/118 [==============================] - 5s 45ms/step - loss: 5.1173 - accuracy: 0.0346 - val_loss: 7.0990 - val_accuracy: 0.0032
Epoch 4/100
118/118 [==============================] - 5s 43ms/step - loss: 4.8100 - accuracy: 0.0537 - val_loss: 7.8615 - val_accuracy: 0.0058
Epoch 5/100
118/118 [==============================] - 5s 43ms/step - loss: 4.5938 - accuracy: 0.0762 - val_loss: 6.6128 - val_accuracy: 0.0058
Epoch 6/100
118/118 [==============================] - 5s 41ms/step - loss: 4.5346 - accuracy: 0.0842 - val_loss: 2160.5320 - val_accuracy: 0.0037
Epoch 7/100
118/118 [==============================] - 5s 41ms/step - loss: 4.1328 - accuracy: 0.1237 - val_loss: 5.5835 - val_accur

118/118 [==============================] - 6s 47ms/step - loss: 0.1505 - accuracy: 0.9589 - val_loss: 9.2485 - val_accuracy: 0.0986
Epoch 58/100
118/118 [==============================] - 6s 48ms/step - loss: 0.0821 - accuracy: 0.9768 - val_loss: 9.0488 - val_accuracy: 0.0981
Epoch 59/100
118/118 [==============================] - 5s 44ms/step - loss: 0.0816 - accuracy: 0.9755 - val_loss: 10.0602 - val_accuracy: 0.0758
Epoch 60/100
118/118 [==============================] - 6s 47ms/step - loss: 0.0672 - accuracy: 0.9806 - val_loss: 9.2624 - val_accuracy: 0.0923
Epoch 61/100
118/118 [==============================] - 5s 45ms/step - loss: 0.1079 - accuracy: 0.9677 - val_loss: 9.5058 - val_accuracy: 0.0917
Epoch 62/100
118/118 [==============================] - 5s 44ms/step - loss: 0.1026 - accuracy: 0.9700 - val_loss: 9.5571 - val_accuracy: 0.0986
Epoch 63/100
118/118 [==============================] - 5s 42ms/step - loss: 0.1000 - accuracy: 0.9699 - val_loss: 9.8288 - val_accuracy: 0.08

In [48]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Make predictions on the test dataset
predictions = model.predict(np.array(X_test).astype('float32'))
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(np.argmax(Y_test, axis=1), predicted_labels)
f1 = f1_score(np.argmax(Y_test, axis=1), predicted_labels, average='weighted')
balanced_acc = balanced_accuracy_score(np.argmax(Y_test, axis=1), predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")

74/74 [==============================] - 2s 15ms/step
Accuracy: 0.08269720101781171
F1 Score: 0.0783020091317103
Balanced Accuracy: 0.08489964528935116


In [ ]:
from tensorflow import keras
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\cub_32_model.h5")
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(np.array(X_train).astype('float32'), Y_train, batch_size=64, epochs=100, validation_split = .2)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
118/118 [==============================] - 14s 65ms/step - loss: 5.8759 - accuracy: 0.0264 - val_loss: 584.3374 - val_accuracy: 0.0058
Epoch 2/100
118/118 [==============================] - 7s 60ms/step - loss: 5.3840 - accuracy: 0.0611 - val_loss: 116.4643 - val_accuracy: 0.0042
Epoch 3/100
118/118 [==============================] - 8s 65ms/step - loss: 5.6141 - accuracy: 0.0457 - val_loss: 438.2541 - val_accuracy: 0.0048
Epoch 4/100
118/118 [==============================] - 7s 57ms/step - loss: 5.4862 - accuracy: 0.0532 - val_loss: 57.9661 - val_accuracy: 0.0064
Epoch 5/100
118/118 [==============================] - 7s 60ms/step - loss: 5.5074 - accuracy: 0.0437 - val_loss: 457.6206 - val_accuracy: 0.0239
Epoch 6/100
118/118 [==============================] - 6s 54ms/step - loss: 4.9445 - accuracy: 0.0714 - val_loss: 12.3051 - val_accuracy: 0.0361
Epoch 7/100
118/118 [==============================] - 6s 53ms/step - loss: 4.8526 - accuracy: 0.0798 - val_loss: 54.1530 - v

118/118 [==============================] - 7s 62ms/step - loss: 0.0689 - accuracy: 0.9832 - val_loss: 9.7158 - val_accuracy: 0.0689
Epoch 58/100
118/118 [==============================] - 6s 49ms/step - loss: 0.1125 - accuracy: 0.9739 - val_loss: 9.6481 - val_accuracy: 0.0732
Epoch 59/100
118/118 [==============================] - 6s 51ms/step - loss: 0.1193 - accuracy: 0.9681 - val_loss: 9.8928 - val_accuracy: 0.0626
Epoch 60/100
118/118 [==============================] - 6s 51ms/step - loss: 0.1924 - accuracy: 0.9479 - val_loss: 10.5270 - val_accuracy: 0.0657
Epoch 61/100
118/118 [==============================] - 6s 49ms/step - loss: 0.2312 - accuracy: 0.9357 - val_loss: 10.3689 - val_accuracy: 0.0705
Epoch 62/100
118/118 [==============================] - 7s 58ms/step - loss: 0.2095 - accuracy: 0.9410 - val_loss: 10.3723 - val_accuracy: 0.0689
Epoch 63/100
118/118 [==============================] - 6s 53ms/step - loss: 0.1901 - accuracy: 0.9453 - val_loss: 10.0452 - val_accuracy: 0

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Make predictions on the test dataset
predictions = new_model.predict(np.array(X_test).astype('float32'))
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(np.argmax(Y_test, axis=1), predicted_labels)
f1 = f1_score(np.argmax(Y_test, axis=1), predicted_labels, average='weighted')
balanced_acc = balanced_accuracy_score(np.argmax(Y_test, axis=1), predicted_labels)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")